In [5]:
import sounddevice as sd
import soundfile as sf
import numpy as np
import scipy.io.wavfile as wav
import speech_recognition as sr
import pyttsx3
import os
import re
import requests
from bs4 import BeautifulSoup

def voice_to_text(audio):
    r = sr.Recognizer()
    with sr.AudioFile(audio) as source:
        audio = r.record(source)
    try:
        return r.recognize_google(audio)
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))
    return (f"Google Speech Recognition could not understand audio")

def record_save_audio(duration_u, file_name):
    fs = 44100
    duration = duration_u
    filename = file_name
    myrecording = sd.rec(duration * fs, samplerate=fs, channels=2, dtype='float64')
    print("Recording Audio")
    sd.wait()
    print("Audio recording complete")
    sf.write(filename, myrecording, fs)

def find_content(text, diction):
    for key, content in diction.items():
        if key.lower() in text.lower():
            return content
    return

def check_if_content_voice(content):
    if content is not None:
        text_to_voice(content)
    else:
        text_to_voice("Can you repeat?")

def text_to_voice(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

def check_if_math(expression):
    pattern = r'(\d+(\.\d+)?)\s*([-+*/])\s*(\d+(\.\d+)?)'
    matches = re.findall(pattern, expression)

    if len(matches) == 0:
        return None

    match = matches[0]
    operand1 = float(match[0])
    operator = match[2]
    operand2 = float(match[3])

    if operator == '+':
        result = operand1 + operand2
    elif operator == '-':
        result = operand1 - operand2
    elif operator == '*':
        result = operand1 * operand2
    elif operator == '/':
        result = operand1 / operand2

    return result

def get_weather():
    url = "https://weather.com/weather/today/l/52.41,16.93?par=google"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    temperature = soup.find(class_="CurrentConditions--tempValue--MHmYY").get_text()
    condition = soup.find(class_="CurrentConditions--phraseValue--mZC_p").get_text()

    return f"The current weather in Poznań is {temperature} and it's {condition}"


In [6]:
greetings = {
    "Hello": "Hello there",
    "Hi": "Hi programmer! How are you?",
    "Howdy": "Howdy to you too, friend!",
    "Good morning": "Good morning! How was your sleep?",
    "Good evening": "Good evening! How was your day?",
    "What's up": "Okay I am fine, how about you?",
    "Goodbye": "Bye bye!"
}

file_name = "output.wav"
record_duration = 5

record_save_audio(record_duration, file_name)
google_audio = voice_to_text(file_name)

content = find_content(google_audio, greetings)

if content is not None:
    text_to_voice(content)
else:
    math_result = check_if_math(google_audio)
    if math_result is not None:
        text_to_voice(f"Your result is: {math_result}")
    else:
        if "weather" in google_audio.lower():
            weather_info = get_weather()
            text_to_voice(weather_info)
        else:
            text_to_voice("Can you repeat?")

Recording Audio
Audio recording complete
